In [ ]:
import main as m
import rasterio as rs
from rasterio.plot import show

PATH = {}
PATH["weather"] = "data/weather/raw/"
PATH["weather-processed"] = "data/weather/processed/"


inp = 'RCM_ta_20112040_ann_p_1_HU.tif'
inp = 'OBS_ta_19912020_JJA_o_0_HU.tif'
inp = 'too-hot-days-19912020_ann_o_0_HU.tif'
inp = 'perception-average-per-month--mm-month-19912020_ann_o_0_HU.tif'
inp = 'average-temperature-year-OBS_ta_19912020_ann_o_0_HU.tif'
inp = 'daily-maximum-perciption--mm-OBS_rx1d_19912020_ann_o_0_HU.tif'
inp = 'wet-days--number-OBS_rr1_19912020_ann_o_0_HU.tif'
#fp = r'inp'
img = rs.open(PATH["weather"]+inp)
show(img)

In [ ]:
### CHATGPT
import rasterio
import geopandas as gpd
from rasterio.features import shapes

# Load the GeoTIFF file into a Rasterio dataset object
with rasterio.open(PATH["weather"]+inp) as src:
    dataset = src.read(1)
    transform = src.transform
    crs = src.crs

# Extract the values from the raster as a list of shapes and values
shapes = shapes(dataset, transform=transform)

shapeRaw = []
for i in shapes:
    shapeRaw.append(i)

# Convert the list of shapes and values to a GeoPandas GeoDataFrame
# gdf = gpd.GeoDataFrame.from_features(shapes, crs=crs)
# gdf = gpd.GeoDataFrame.from_features(shapeRaw, crs=crs)

# Drop any missing or invalid values
# gdf.dropna(inplace=True)

# y is a value
# x is a polygon
polygons = []
values = []
data = []
for x,y in shapeRaw:
    l = []
    l.append(x['coordinates'])
    #print(i['coordinates'])
    #print(i['type'])
    l.append(y)
    data.append(l)
df = m.pd.DataFrame(data, columns=['geometry', 'value'])
# depemnds on, some values could be integers, some of the floats. Should be checked.

df.value = df.value.round(0) # rounding values. astype(int) doesnt save space for parquet files 

#df.value = df.value.astype(int) # integer save space, float = not
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=[m.Polygon(x[0]) for x in df['geometry']])
t = gdf.to_crs({'init': 'epsg:4326'}) #4326 works since https://epsg.io/?q=4326
t = t[t.value > min(t.value)]

In [ ]:
import folium
import matplotlib
import mapclassify
display(t.explore("value", cmap="YlGn"))
t.to_parquet(PATH["weather-processed"]+inp+".parquet")